
<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Random Forests and ExtraTrees


_Authors: Matt Brems (DC), Riley Dallas (AUS)_

---

## Random Forests
---

With bagged decision trees, we generate many different trees on pretty similar data. These trees are **strongly correlated** with one another. Because these trees are correlated with one another, they will have high variance. Looking at the variance of the average of two random variables $T_1$ and $T_2$:

$$
\begin{eqnarray*}
Var\left(\frac{T_1+T_2}{2}\right) &=& \frac{1}{4}\left[Var(T_1) + Var(T_2) + 2Cov(T_1,T_2)\right]
\end{eqnarray*}
$$

If $T_1$ and $T_2$ are highly correlated, then the variance will about as high as we'd see with individual decision trees. By "de-correlating" our trees from one another, we can drastically reduce the variance of our model.

That's the difference between bagged decision trees and random forests! We're going to do the same thing as before, but we're going to de-correlate our trees. This will reduce our variance (at the expense of a small increase in bias) and thus should greatly improve the overall performance of the final model.

So how do we "de-correlate" our trees?

Random forests differ from bagging decision trees in only one way: they use a modified tree learning algorithm that selects, at each split in the learning process, a **random subset of the features**. This process is sometimes called the *random subspace method*.

The reason for doing this is the correlation of the trees in an ordinary bootstrap sample: if one or a few features are very strong predictors for the response variable (target output), these features will be used in many/all of the bagged decision trees, causing them to become correlated. By selecting a random subset of features at each split, we counter this correlation between base trees, strengthening the overall model.

For a problem with $p$ features, it is typical to use:

- $\sqrt{p}$ (rounded down) features in each split for a classification problem.
- $p/3$ (rounded down) with a minimum node size of 5 as the default for a regression problem.

While this is a guideline, Hastie and Tibshirani (authors of Introduction to Statistical Learning and Elements of Statistical Learning) have suggested this as a good rule in the absence of some rationale to do something different.

Random forests, a step beyond bagged decision trees, are **very widely used** classifiers and regressors. They are relatively simple to use because they require very few parameters to set and they perform pretty well.
- It is quite common for interviewers to ask how a random forest is constructed or how it is superior to a single decision tree.

--- 

## Extremely Randomized Trees (ExtraTrees)
Adding another step of randomization (and thus de-correlation) yields extremely randomized trees, or _ExtraTrees_. Like Random Forests, these are trained using the random subspace method (sampling of features). However, they are trained on the entire dataset instead of bootstrapped samples. A layer of randomness is introduced in the way the nodes are split. Instead of computing the locally optimal feature/split combination (based on, e.g., information gain or the Gini impurity) for each feature under consideration, a random value is selected for the split. This value is selected from the feature's empirical range.

This further reduces the variance, but causes an increase in bias. If you're considering using ExtraTrees, you might consider this to be a hyperparameter you can tune. Build an ExtraTrees model and a Random Forest model, then compare their performance!

That's exactly what we'll do below.

## Import libraries
---

We'll need the following libraries for today's lecture:
- `pandas`
- `numpy`
- `GridSearchCV`, `train_test_split` and `cross_val_score` from `sklearn`'s `model_selection` module 
- `RandomForestClassifier` and `ExtraTreesClassifier` from `sklearn`'s `ensemble` module 

In [259]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

## Load Data
---

Load `train.csv` and `test.csv` from Kaggle into `DataFrames`.

In [230]:
train, test = pd.read_csv("datasets/train.csv"), pd.read_csv("datasets/test.csv")

In [231]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


## Data Cleaning: Drop the two rows with missing `Embarked` values from train
---

## Data Cleaning: `Fare`
---

The test set has one row with a missing value for `Fare`. Fill it with the average `Fare` with everyone from the same `Pclass`. **Use the training set to calculate the average!**

In [232]:
test[test['Fare'].isnull()].Pclass.iloc[0]

3

In [233]:
def impute_missing_fare(train, test):
        '''Input is a dataframe with a single missing value in the Fare column.
           Output is the dataframe with the average fare for the a ppassenger of that class '''
        missing_class = test[test['Fare'].isnull()].Pclass.iloc[0]
        missing_row = test[test['Fare'].isnull()].Pclass.index
#         print(missing_class)
        missing_fare = train[train["Pclass"] == missing_class]['Fare'].mean()
#         print(missing_fare)
#         print(test[test['Fare'].isnull()]['Fare'])
        test.loc[missing_row, 'Fare'] = missing_fare
#         print(test.loc[missing_row, 'Fare'])

## Data Cleaning: `Age`
---

Let's simply impute all missing ages to be **999**. 

**NOTE**: This is not a best practice. However, 
1. Since we haven't really covered imputation in depth
2. And the proper way would take too long to implement (thus detracting) from today's lecture
3. And since we're ensembling with Decision Trees

We'll do it this way as a matter of convenience.

In [234]:
def fill_age(train, value):
    train['Age'].fillna(value, inplace=True)

In [235]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Feature Engineering: `Cabin`
---

Since there are so many missing values for `Cabin`, let's binarize that column as follows:
- 1 if there originally was a value for `Cabin`
- 0 if it was null

**Do this for both `train` and `test`**

In [236]:
def cabin_fill(train):
    train['Cabin'] = train.Cabin.fillna(0).map(lambda cab : 1 if cab != 0 else cab)

## Feature Engineering: Dummies
---

Dummy the `Sex` and `Embarked` columns. Be sure to set `drop_first=True`.

In [239]:
impute_missing_fare(train, test)
fill_age(train, 999)
fill_age(test, 999)
cabin_fill(train)
cabin_fill(test)
train = pd.get_dummies(train, columns = ['Embarked', 'Sex'],  drop_first=True)
test = pd.get_dummies(test, columns = ['Embarked', 'Sex'],  drop_first=True)

## Model Prep: Create `X` and `y` variables
---

Our features will be:

```python
features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Sex_male', 'Embarked_Q', 'Embarked_S']
```

And our target will be `Survived`

In [240]:
features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Sex_male', 'Embarked_Q', 'Embarked_S']

In [242]:
target = "Survived"

In [248]:
X, y = train[features], train[target]

## Challenge: What is our baseline accuracy?
---

The baseline accuracy is the percentage of the majority class, regardless of whether it is 1 or 0. It serves as the benchmark for our model to beat.

In [251]:
y.value_counts(normalize=True)

0    0.616162
1    0.383838
Name: Survived, dtype: float64

## Train/Test Split
---

I know it can be confusing having an `X_test` from our training data vs a test set from Kaggle. If you want, you can use `X_val`/`y_val` for what we normally call `X_test`/`y_test`.

In [252]:
X_train, X_val, y_train, y_val  = train_test_split(X, y, stratify= y, random_state = 42)

## Model instantiation
---

Create an instance of `RandomForestClassifier` and `ExtraTreesClassifier`.

In [279]:
rf = RandomForestClassifier()
et = ExtraTreesClassifier()

In [281]:
base_rf = rf.fit(X_train, y_train)
print(base_rf.score(X_train, y_train))
base_rf.score(X_val, y_val)

0.9850299401197605


0.7533632286995515

In [283]:
#Baseline
1-y_val.sum()/len(y_val)

0.6143497757847534

## Model Evaluation
---

Which one has a higher `cross_val_score`?

In [255]:
rf_cv = cross_val_score(rf, X_train, y_train, cv = 5).mean()

In [254]:
et_cv = cross_val_score(et, X_train, y_train, cv = 5).mean()

In [256]:
rf_cv

0.820379306475143

In [257]:
et_cv

0.7994388957468297

## Grid Search
---

They're both pretty close performance-wise. We could Grid Search over both, but for the sake of time we'll go with `RandomForestClassifier`.

In [287]:
param_grid = {
    'n_estimators': [50, 100, 250, 500, 1000],
    'max_depth': [2, 3, 5, 7, 10, None],
    'max_features': ['sqrt', 'log2', 1/3]   
}

In [293]:
gs = GridSearchCV(rf, param_grid= param_grid, cv =5, n_jobs=-1,  verbose=1)

In [294]:
%%time
gs.fit(X_train, y_train)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:   29.8s


CPU times: user 1.23 s, sys: 301 ms, total: 1.53 s
Wall time: 33 s


[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:   32.9s finished


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [2, 3, 5, 7, 10, None],
                         'max_features': ['sqrt', 'log2', 0.3333333333333333],
                         'n_estimators': [50, 100, 250, 500, 1000]},
             verbose=1)

In [295]:
gs.best_score_

0.8413533834586466

In [296]:
gs.best_params_

{'max_depth': 10, 'max_features': 'log2', 'n_estimators': 50}

In [297]:
best_rf = gs.best_estimator_.fit(X_train, y_train)
print(best_rf.score(X_train, y_train))
best_rf.score(X_val, y_val)


0.9476047904191617


0.7757847533632287

In [302]:
[X_train.columns,best_rf.feature_importances_]

[Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Sex_male',
        'Embarked_Q', 'Embarked_S'],
       dtype='object'),
 array([0.07675495, 0.20996207, 0.04951557, 0.04549424, 0.22373586,
        0.07507961, 0.28596014, 0.00784496, 0.02565261])]

In [310]:
pd.DataFrame({'feature': X_train.columns,'importance': best_rf.feature_importances_}).sort_values(by = 'importance',ascending=False)

,feature,importance
6,Sex_male,0.285960
4,Fare,0.223736
1,Age,0.209962
0,Pclass,0.076755
5,Cabin,0.075080
2,SibSp,0.049516
3,Parch,0.045494
8,Embarked_S,0.025653
7,Embarked_Q,0.007845


## Kaggle Submission
---

Now that we've evaluated our model, let's submit our predictions to Kaggle.